# Exercise 8


In [8]:
import numpy as np

In [9]:
# quick helper to bootstrap any statistic
def get_bootstrap(data, func, n_rep=5000):
    n = len(data)
    reps = np.empty(n_rep)
    for i in range(n_rep):
        samp = np.random.choice(data, size=n, replace=True)
        reps[i] = func(samp)
    return reps

In [10]:
# Exercise 13: estimate P(a < mean - mu < b)
def solve_ex13(vals, a=-5, b=5, reps=5000):
    arr = np.array(vals)
    # use sample mean as stand-in for true mu
    mu_est = arr.mean()
    # bootstrap means
    bs_means = get_bootstrap(arr, np.mean, reps)
    # center and check bounds
    diffs = bs_means - mu_est
    prob = np.mean((diffs > a) & (diffs < b))
    return prob

In [11]:
# Exercise 15: bootstrap var of sample variance
def solve_ex15(vals, reps=5000):
    arr = np.array(vals)
    # sample variance function
    def var1(x): return x.var(ddof=1)
    bs_s2 = get_bootstrap(arr, var1, reps)
    # return variance of the s2 replicates
    return bs_s2.var(ddof=1)

In [12]:
# Exercise 3: Pareto sample, get mean & median precision
def solve_ex3(n=200, beta=1, alpha=1.05, reps=5000):
    # numpy.pareto gives Y ~ Pareto(alpha) on (0, inf), so shift+scale
    Y = np.random.pareto(alpha, size=n)
    X = beta * (1 + Y)

    mean_val = X.mean()
    med_val = np.median(X)

    bs_means = get_bootstrap(X, np.mean, reps)
    bs_meds = get_bootstrap(X, np.median, reps)
    var_mean = bs_means.var(ddof=1)
    var_med = bs_meds.var(ddof=1)

    return {
        'mean': mean_val,
        'median': med_val,
        'var_mean_bs': var_mean,
        'var_med_bs': var_med,
        'bs_means': bs_means,
        'bs_meds': bs_meds
    }

In [13]:
np.random.seed(42)

# Ex 13 data
data13 = [56, 101, 78, 67, 93, 87, 64, 72, 80, 69]
p_est = solve_ex13(data13, a=-5, b=5)
print(f"[Ex13] Estimated P = {p_est:.3f}")

# Ex 15 data
data15 = [5,4,9,6,21,17,11,20,7,10,21,15,13,16,8]
var_s2 = solve_ex15(data15)
print(f"[Ex15] Bootstrap Var(S^2) ≈ {var_s2:.4f}")

# Ex 3 simulation
res3 = solve_ex3(n=200, beta=1, alpha=1.05)
print("[Ex3] Pareto sample stats:")
print(f"  Mean = {res3['mean']:.4f}, Median = {res3['median']:.4f}")
print(f"  BS Var(mean) = {res3['var_mean_bs']:.6f}")
print(f"  BS Var(med)  = {res3['var_med_bs']:.6f}")

# compare precision
better = 'mean' if res3['var_mean_bs'] < res3['var_med_bs'] else 'median'
print(f"  => {better.capitalize()} more precise (smaller bootstrap var)")

[Ex13] Estimated P = 0.768
[Ex15] Bootstrap Var(S^2) ≈ 59.4683
[Ex3] Pareto sample stats:
  Mean = 4.2538, Median = 1.9175
  BS Var(mean) = 0.502776
  BS Var(med)  = 0.018311
  => Median more precise (smaller bootstrap var)
